In [1]:
!pip3 install keybert

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 86.0/86.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 239.0/239.0 kB 7.4 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 7.9 MB/s eta 0:00:00
     ---------------------------------------- 84.5/84.5 kB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 8.9 MB/s eta 0:00:00
     -------------------------------------- 162.5/162.5 MB 7.4 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 8.7 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 8.0 MB/s eta 0:00:00
     ------------------------------------- 190.3/190.3 kB 12.0 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 9.2 MB/s eta 0:00:00
  Crea

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.


In [2]:
from keybert import KeyBERT
import pickle
import re
from tqdm import tqdm
import pandas as pd
import os

In [3]:
def doc2key(cleaned_content):
    
    kw_model = KeyBERT(model='all-MiniLM-L6-v2')
    keywords = kw_model.extract_keywords(doc)
    
    n2_kwd = kw_model.extract_keywords(cleaned_content, keyphrase_ngram_range=(2, 2), stop_words='english',
                                  use_mmr=True, diversity=0.7, top_n=5)

    n1_kwd = kw_model.extract_keywords(cleaned_content, keyphrase_ngram_range=(1, 1), stop_words='english',
                                  use_mmr=True, diversity=0.7, top_n=10)
    for idx,i in enumerate(n2_kwd):
        n2_kwd[idx] = i[0]
    for idx,i in enumerate(n1_kwd):
        n1_kwd[idx] = i[0]  

    n1_kwd.extend(n2_kwd)
    kwd = n1_kwd
    return kwd

In [4]:
from_date = '2023-1-30'
to_date = '2023-1-31'
searching_word = 'metaverse'

In [5]:
name = "News{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')

In [6]:
cwd = os.getcwd()

In [7]:
main_path = cwd + '/' + searching_word

In [8]:
with open(main_path + '/' + 'news/' + name, 'rb') as f:
    data = pickle.load(f)

In [9]:
data = data.reset_index()

In [10]:
data = data.rename({"index":"date"},axis = 1)

In [11]:
idx = []
for x in data['date']:
    tmp = x.split('/')
    if len(tmp[0]) == 1:
        tmp[0] = '0'+tmp[0]
    if len(tmp[1]) == 1:
        tmp[1] = '0'+tmp[1]
    tmp = tmp[2] + tmp[0] + tmp[1]
    idx.append(''.join(tmp))

In [12]:
data['date'] = idx
data = data.rename({'date':'일자'},axis = 1)

In [13]:
kwd_list = []
for docs in tqdm(data['news']):
    kwds = []
    for doc in docs:
        try:
            cleaned_content = re.sub(r'[^\.\?\!\w\d\s]','',doc) # 문장단위로 끊기
            cleaned_content = cleaned_content.replace('\n',' ')
            cleaned_content = cleaned_content.lower()
            kwd = doc2key(cleaned_content)
            kwds.append(kwd)
        except:
            print(doc)
    kwd_list.append(kwds)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:29<00:00, 14.63s/it]


In [14]:
data['kwd'] = kwd_list

In [15]:
name = "Keyword{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')

In [142]:
with open(main_path + '/' + 'keyword/' + name,'wb') as f:
    pickle.dump(data,f)

In [143]:
data['kwd'][0][0]

['tokens',
 'floki',
 'trillion',
 'bridge',
 'calamitous',
 'stability',
 'relocation',
 'disclaimer',
 'rationale',
 'launching',
 'burning tokens',
 'floki primary',
 'risks bridges',
 'disappeared crosschain',
 'pricing ratio']